# kinetic proofreading animation
PCN 8/2017<br>
This code just displays the results from riboproof2.py, via the data file kproof.npz<br>
tRNA-AA-GTP complexes appear suddenly from solution<br>
red = wrong; green = right<br>
GTP = dark color; GDP = bright color<br>
GTP = cylinder; GDP = sphere; <br>
incorporated = sphere.<br>
GTP hydrolysis = shift right and deepening color<br>
rejected tRNA-AA-GxP appear briefly ghosted then disappear<br>
AA incorporation shifts the chain over to the right

In [1]:
from vpython import *
from numpy.random import random as rng
import numpy as np
fracshow = 0.94 
framerate = 30              # frame rate in Hz, so 30 means 33ms per frame
duration = 1500             # desired duration of movie [s]
mysphererad = 0.15
hydrolysis_sh = mysphererad-.05
data = np.load('kproof.npz')
ts = data['ts'][0]          # transition times
states = data['states'][0]  # sequence of states
Nstates = len(states)
timeunit = duration/ts[-1]  # real time [s] per unit in the simulation
rejectduration = 2          # how long to display rejected (in frames)
timerT = 0; timerD = 0      # initialize
framerate*duration*fracshow # how many frames we'll see

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

42300.0

Time is initially 0. State is initially states[0]=0. At time ts[0] transition to states[1], which is negative if it terminates with an incorporation.

In [2]:
scene = canvas(title='KP')
scene.forward = vector(0,.4,-1) # camera viewpoint
# represent ribosome by a blue box
static = box(pos=vector(0,0,0), size=vector(1,1,.2), color=color.blue)
bindsite = vector(-.15,0,.1)
# represent the bound tRNA-GTP complex by a cylinder: invisible if none bound, dark
#  green if right, dark red if wrong
boundT = cylinder(pos=bindsite+vector(0,0,mysphererad*1.5), axis=vector(0,0,.17), radius=mysphererad*.85)
boundT.visible = False # initially none
# after a tRNA-GTP is rejected, briefly show it as ghost
rejectT = cylinder(pos=bindsite+vector(mysphererad,0,2*mysphererad*1.5), axis=vector(0,0,.17), \
                   radius=mysphererad*.85, opacity=0.3)
rejectT.visible = False
# represent bound tRNA-GDP complex by a sphere, with bright color
boundD = sphere(pos=bindsite+vector(hydrolysis_sh,0,.3), radius=mysphererad)
boundD.visible = False # initially none
chain = [] # list of blobs representing incorporated AAs. Last item in chain is most recent addition.
chainsite = vector(+.25,0,.1)
# after a tRNA-GDP is rejected, briefly show it as ghost
rejectD = sphere(pos=bindsite+vector(mysphererad+hydrolysis_sh,0,.3+1.5*mysphererad), \
                 radius=mysphererad, opacity=0.3)
rejectD.visible = False

<IPython.core.display.Javascript object>

In [ ]:
j=0;  # which event we're on now
for whichframe in range(int(framerate*duration*fracshow)): # stop slightly short of end
    timenow = whichframe/framerate
    incorp = False                           # initialize
    while ts[j]*timeunit < timenow:          # update which event we're on now
        if states[j]==-3:                    # elongate chain: correct incorporation
            rejectD.visible = False; rejectT.visible = False
            for amino in chain:              # push everybody to right
                amino.pos = amino.pos + vector(mysphererad*2,0,0)
            chain = chain + [sphere(pos=chainsite+vector(0,0,.3), radius=mysphererad,color=vector(0,1,0))]
        elif states[j]==-4:                  # elongate chain: wrong incorporation
            rejectD.visible = False; rejectT.visible = False
            for x in chain:
                x.pos = x.pos + vector(mysphererad*2,0,0)
            chain = chain + [sphere(pos=chainsite+vector(0,0,.3), radius=mysphererad,color=vector(1,0,0))]
        j+=1
        if states[j]==0 and states[j-1]==1:  # correct.T unbinds: dark translucent green
            rejectT.visible = True           #later will also turn off boundT
            timerT = 0                       # display rejected amino for a while 
            rejectT.color = vector(.5,1,.5)
        elif states[j]==0 and states[j-1]==2:# wrong.T unbinds: dark translucent red
            rejectT.visible = True           #later will also turn off boundT
            timerT = 0                       # display rejected amino for a while 
            rejectT.color = vector(1,.5,.5)
        elif states[j]==0 and states[j-1]==3:# correct.D unbinds: bright translucent green
            rejectD.visible = True           #later will also turn off boundD
            timerD = 0                       # display rejected amino for a while 
            rejectD.color = vector(0,1,0)
        elif states[j]==0 and states[j-1]==4:# wrong.D unbinds: bright translucent red
            rejectD.visible = True           #later will also turn off boundD
            timerD = 0                       # display rejected amino for a while 
            rejectD.color = vector(1,0,0)

    if states[j]==0:                         # unoccupied; continue to display rejected if there is one
        boundT.visible = False
        boundD.visible = False
    elif states[j]==1:                       # correct tRNA.GTP: dark green
        boundT.visible = True
        boundT.color = vector(0,.3,0)
        boundD.visible = False
        if timerD > rejectduration: rejectD.visible = False
        if timerT > rejectduration: rejectT.visible = False
    elif states[j]==2:                       # wrong tRNA.GTP: dark red
        boundT.visible = True
        boundT.color = vector(.3,0,0)
        boundD.visible = False
        if timerD > rejectduration: rejectD.visible = False
        if timerT > rejectduration: rejectT.visible = False
    elif states[j]==3 or states[j]==-3:      # correct tRNA.GDP: green
        boundT.visible = False
        boundD.visible = True
        boundD.color = vector(0,1,0)
        if timerD > rejectduration: rejectD.visible = False
        if timerT > rejectduration: rejectT.visible = False
    elif states[j]==4 or states[j]==-4:      # wrong tRNA.GDP: red
        boundT.visible = False
        boundD.visible = True
        boundD.color = vector(1,0,0)
        if timerD > rejectduration: rejectD.visible = False
        if timerT > rejectduration: rejectT.visible = False
    timerT += 1; timerD += 1
    rate(framerate)                          # pause till it's time to show the frame

In [ ]:
states[:5]

In [ ]:
ts[:5]

x